In [16]:
import numpy as np
import pandas as pd
import gzip
import kipoiseq
import importlib
import Models
importlib.reload(Models)
%matplotlib
import matplotlib

Using matplotlib backend: MacOSX


In [2]:
# output cancer gene list

def get_cons(*items, method='inter'):
    """return the intersection or union of multiple 
    lists. method =inter / union 
    output type: set"""
    
    # make them all the set types
    set_items=[]
    for i, genes in enumerate(items):
        if len(genes) == len(set(genes)):
            set_items.append(set(genes))
        else:
            return('WARNING: the {}th gene set contains duplicate genes.'.format(i+1))
            
    num = len(items) 
    cur_list = []
    cur_index = None
    for i, genes in enumerate(set_items):
        if cur_index == None: 
            cur_list = genes; cur_index = i
            continue
        else:
            if method == 'inter': cur_list = cur_list.intersection(genes)
            if method == 'union': cur_list = cur_list.union(genes)
            cur_index = i
    return(cur_list)
    
    

In [3]:

def gene_extractor(gff_file, mapping_list, gzziped=False):
    """input: gff file, list of wanted gene names
       output: dict
    """
    def _open(file):
        return gzip.open(file, 'rt') if gzziped else open(file,'r')
    def get_nonoverlap(chr_, start, end, gene_pos):
        """ return the end position of the last up stream 
        non-overlapping gene"""
        return(max(np.array(gene_pos[chr_])[:,1][np.array(gene_pos[chr_])[:,1] < start]))
        
    with _open(gff_file) as f:
        up_gene = False; gene_pos= {} # {(start, end), ...}
        last_gene = None; cur_gene = None
        gene_dict={}
        for line in f:
            if line.startswith('#'): continue
            chr_, xx, seq_type, start, end, score, strand, frame, attri = line.strip().split('\t')
            if attri.split(';')[0].startswith('ID=gene-'):
                cur_gene = (attri.split(';')[0].split('ID=gene-')[1], start, end)
                
                if cur_gene[0] in mapping_list and last_gene != None: 
                    #store upstream and current gene
                    gene_dict['up_border'] = get_nonoverlap( chr_, int(start), 
                                                           int(end), gene_pos)
                    gene_dict['name'] = cur_gene[0]
                    gene_dict['seq_type'] = seq_type
                    gene_dict['chr'] = chr_
                    gene_dict['start'] = start
                    gene_dict['end'] = end
                    gene_dict['strand'] = strand
                    gene_dict['attri'] = attri
                    up_gene = True
                elif up_gene and  int(start) > int(gene_dict['end']):
                    #store downstread gene (only the start of downstrad gene
                    #                       is bigger than the end of target
                    #                       gene)
                     
                    gene_dict['down_border'] = start
                    up_gene = False
                    yield gene_dict
                    gene_dict = {}
                
                gene_pos.setdefault(chr_,[]).append((int(start),int(end)))
                last_gene=cur_gene
            

In [4]:

dam_file='/Users/jerryliu/Documents/Vu_uva/internship/CCLE/\
mutation_files/CCLE_mutations_bool_damaging.csv'
hot_file='/Users/jerryliu/Documents/Vu_uva/internship/CCLE/\
mutation_files/CCLE_mutations_bool_hotspot.csv'
non_file='/Users/jerryliu/Documents/Vu_uva/internship/CCLE/\
mutation_files/CCLE_mutations_bool_nonconserving.csv'
con_file='/Users/jerryliu/Documents/Vu_uva/internship/CCLE/\
mutation_files/CCLE_mutations_bool_otherconserving.csv'
Con_file='/Users/jerryliu/Documents/Vu_uva/internship/CCLE/\
mutation_files/Census_allSat.csv'


In [5]:
gene_lists =  [ pd.read_csv(path).columns[1:].map(lambda x:x.split(' ')[0])  
    for path in [dam_file, hot_file, non_file] ]

Con_genes = pd.read_csv(Con_file).loc[:,'Gene Symbol']

final_genes = get_cons(get_cons(*gene_lists, method='union'), Con_genes, method= 'inter')

In [20]:
# extract the positions of each cancer gene
gff_file = '/Users/jerryliu/jerry_jupyter/internship/files/GRCh37_latest_genomic.gff.gz'
vcf_file = '/Users/jerryliu/jerry_jupyter/internship/files/SRR8788980'
gene_pos = Models.gene_extractor(gff_file = gff_file, mapping_list=final_genes, gzziped=True)
transform_path = 'gs://dm-enformer/models/enformer.finetuned.SAD.robustscaler-PCA500-robustscaler.transform.pkl'
model_path = 'https://tfhub.dev/deepmind/enformer/1'
fasta_file = '/Users/jerryliu/jerry_jupyter/internship/files/Homo_sapiens_assembly19.fasta'
# Download targets from Basenji2 dataset 
targets_txt = 'https://raw.githubusercontent.com/calico/basenji/master/manuscripts/cross2020/targets_human.txt'
df_targets = pd.read_csv(targets_txt, sep='\t')
TSS_file = '/Users/jerryliu/jerry_jupyter/internship/files/mart_export.txt.gz'

In [26]:
# counts = 0
# biggest = 0
# for i,gene in enumerate(gene_pos):
#     length =(int(gene['end']) - int(gene['start'])) +1
#     biggest  =  length  if biggest < length else biggest
#     if length > SEQUENCE_LENGTH: counts += 1
#     #if int(gene['start']) - int(gene['end']) >= SEQUENCE_LENGTH: counts += 1
# #print(biggest)
# print(counts
len(final_genes)

710

In [32]:
!ls ../files
!pwd
36271661 - 35214822

GRCh37_latest_genomic.gff.gz       VCFv4.1.pdf
Homo_sapiens_assembly19.fasta      VCFv4.3.pdf
Homo_sapiens_assembly19.fasta.dict clinvar.vcf
Homo_sapiens_assembly19.fasta.fai  output
SAMv1.pdf                          未命名.ipynb
SRR8788980.1.new.13.output.g.vcf
/Users/jerryliu/jerry_jupyter/internship/code


1056839

In [87]:
SEQUENCE_LENGTH = 393216
counts = 0
for i, gene in enumerate(gene_pos): 
    if gene['chr'] == '13' and \
    (int(gene['down_border']) - int(gene['up_border'])) < SEQUENCE_LENGTH:
        print(gene)
        counts += 1
    if counts >5: break    

{'up_border': 20437789, 'name': 'ZMYM2', 'seq_type': 'gene', 'chr': '13', 'start': '20531563', 'end': '20663255', 'strand': '+', 'attri': 'ID=gene-ZMYM2;Dbxref=GeneID:7750,HGNC:HGNC:12989,MIM:602221;Name=ZMYM2;description=zinc finger MYM-type containing 2;gbkey=Gene;gene=ZMYM2;gene_biotype=protein_coding;gene_synonym=FIM,MYM,RAMP,SCLL,ZNF198', 'down_border': '20676840'}
{'up_border': 21536407, 'name': 'LATS2', 'seq_type': 'gene', 'chr': '13', 'start': '21547175', 'end': '21635725', 'strand': '-', 'attri': 'ID=gene-LATS2;Dbxref=GeneID:26524,HGNC:HGNC:6515,MIM:604861;Name=LATS2;description=large tumor suppressor kinase 2;gbkey=Gene;gene=LATS2;gene_biotype=protein_coding;gene_synonym=KPM', 'down_border': '21668434'}
{'up_border': 28529500, 'name': 'CDX2', 'seq_type': 'gene', 'chr': '13', 'start': '28535055', 'end': '28543452', 'strand': '-', 'attri': 'ID=gene-CDX2;Dbxref=GeneID:1045,HGNC:HGNC:1806,MIM:600297;Name=CDX2;description=caudal type homeobox 2;gbkey=Gene;gene=CDX2;gene_biotype=pr

In [7]:
model = Models.Enformer(model_path)

fasta_extractor = Models.FastaStringExtractor(fasta_file)

In [54]:
print('cancer gene length: '+str(36246873-35516407))
print('gene length + non-coding part: '+str(36271661-35214822))

cancer gene length: 730466
gene length + non-coding part: 1056839


In [130]:
up_border = 20437789; down_border = 20676840
Interval = kipoiseq.Interval('13', 20526481,  20526481).resize(SEQUENCE_LENGTH)  # @param
seq_extractor = kipoiseq.extractors.VariantSeqExtractor(reference_sequence=fasta_extractor)
center = Interval.center() - Interval.start

reference = seq_extractor.extract(Interval, [], anchor=center)

reference_prediction = model.predict_on_batch(Models.one_hot_encode(reference)[np.newaxis])['human'][0]


tracks = {'CAGE/neutrofils ref': reference_prediction[:, 4767],
          'CHIP:H3K27ac:neutrophil ref': reference_prediction[:, 2280],
          'DNASE:CD14-positive monocyte female': reference_prediction[:, 41],
          'DNASE:keratinocyte female': reference_prediction[:, 42],
          'CHIP:H3K27ac:keratinocyte female': reference_prediction[:, 706],
          'CAGE:Keratinocyte - epidermal': np.log10(1 + reference_prediction[:, 4799])}
Models.plot_tracks(tracks, Interval.resize(reference_prediction.shape[0] * 128), height = 1.5)

#Models.plot_full_track(tracks, Interval, up_border, down_border, height=1.5)

In [152]:
reference_prediction.shape

(896, 5313)

In [24]:
# generate prediction for all genes
SEQUENCE_LENGTH = 393216
acc_id = 'SRR8788980'
gene_pos = Models.gene_extractor(gff_file = gff_file, mapping_list=final_genes, gzziped=True)
for gene_dict in gene_pos:
    # create Gene object
    gene = Models.Gene(**gene_dict)
    # create TSS interval objects list
    TSS_list = [kipoiseq.Interval(gene.chrom, TSS,  TSS).resize(SEQUENCE_LENGTH) \
     for TSS in Models.extract_TSS(gene_id = gene.name, TSS_file = TSS_file ) ]

    # prepare sequence for each TSS
    seq_extractor = kipoiseq.extractors.VariantSeqExtractor(reference_sequence=fasta_extractor)
    
    expression = np.zeros(5313)
    for TSS_item in TSS_list:
        reference  = seq_extractor.extract(TSS_item, [],
                                           anchor = TSS_item.center() - TSS_item.start)
        # generate variants list for each TSS
        variants = [i for i in  Models.variants_extractor(vcf_path = vcf_file, 
                                  Interval = TSS_item,acc_id = acc_id, gzipped=True) ]
        
        alternate = seq_extractor.extract(TSS_item, variants, 
                                          anchor = TSS_item.center() - TSS_item.start )
        # make prediction on the 114,688-bp centering the TSS of the gene
        reference_prediction = model.predict_on_batch(Models.one_hot_encode(reference)[np.newaxis])['human'][0]
        alternate_prediction = model.predict_on_batch(Models.one_hot_encode(alternate)[np.newaxis])['human'][0]
        
        # 1* 5313
        expression = expression + Models.calculate_TSS(ref_pre = reference_prediction, alt_pre = alternate_prediction)
    print(expression)
    
    break
        

[-0.3422581  -0.28567305 -0.40039402 ...  0.05264773  0.57751823
  0.81826293]
